In [9]:
import numpy as np
import pandas as pd 
import os
import sys
import importlib

In [10]:
PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
RESOURCES_DIR = os.path.join(PROJECT_DIR, 'resources')
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')

sys.path.append(PROJECT_DIR)

In [11]:
from scripts.utils import prepare_lut
fs_default = prepare_lut()

In [12]:
template = pd.read_excel(f'{RESOURCES_DIR}/template_cortical_thickness.xlsx')

In [13]:
import scripts.utils
importlib.reload (scripts.utils)
from scripts.utils import get_raw_thickness
gmv = get_raw_thickness(RAW_DATA_DIR)

In [14]:
demo1= pd.read_excel(f'{DATA_DIR}/demographics/Demographics_data.xlsx')
demo2= pd.read_excel(f'{DATA_DIR}/demographics/Demographics_Test_data_2025.08.08.xlsx')

In [15]:
sub2 = np.unique(demo2['SubID'])
demo2 = demo2[(demo2['SubID'].isin(sub2)) & (demo2['Wave'] == 'Baseline')].copy()

In [16]:
columns_to_keep2 = ['ScanID', 'Sex', 'Age']
demo2 = demo2[columns_to_keep2]
demo2.rename(columns={'ScanID': 'pid', 'Sex': 'sex', 'Age': 'age'}, inplace=True)


In [17]:
columns_to_keep1 = ['No', 'Sex', 'Age']
demo1 = demo1[columns_to_keep1]
demo1.rename(columns={'No': 'pid', 'Sex': 'sex', 'Age': 'age'}, inplace=True)


In [18]:
demo = pd.concat([demo1, demo2])
demo['pid'] = demo['pid'].astype('str')
demo.set_index('pid', inplace=True)

In [19]:
demo_and_gmv = pd.concat([demo, gmv.loc[demo.index]], axis=1, join='inner')

In [22]:
from scripts.utils import adjust_thick_template

m, f = adjust_thick_template(demo_and_gmv, template)

m.to_excel(f'{DATA_DIR}/processed/male_chinese_all_cortical_thick.xlsx', index=False)
f.to_excel(f'{DATA_DIR}/processed/female_chinese_all_cortical_thick.xlsx', index=False)

In [ ]:
# Selecting only healthy controls
hc = ['1855', '1856', '1857', '1858', '1859', '1860', '1861', '1862', '1863', '1864']

In [ ]:
hc_f = [patient for patient in hc if patient in f['SubjectID'].values]
hc_m = [patient for patient in hc if patient in m['SubjectID'].values]

In [ ]:
m[m['SubjectID'].isin(hc_m)].to_excel(f'{DATA_DIR}/processed/male_chinese_hc_cortical_thick.xlsx', index=False)
f[f['SubjectID'].isin(hc_f)].to_excel(f'{DATA_DIR}/processed/female_chinese_hc_cortical_thick.xlsx', index=False)

In [66]:
# Selecting only the patients
pat_f = [patient for patient in f['SubjectID'].values if patient not in hc]
pat_m = [patient for patient in m['SubjectID'].values if patient not in hc]
m[m['SubjectID'].isin(pat_m)].to_excel(f'{DATA_DIR}/processed/male_chinese_pat_cortical_thick.xlsx', index=False)
f[f['SubjectID'].isin(pat_f)].to_excel(f'{DATA_DIR}/processed/female_chinese_pat_cortical_thick.xlsx', index=False)

In [67]:
pat_m

['1198', '1234', '1273', '2015052501', '2015060902', '2019012101']